In [1]:
import modules
from modules import Aqua_Obj
from optimization import OpData
import optimization
from utilities import *
import copy
import numpy as np
from gis.gis_handler import GISHandler


max_iter = 2000    

def env_opt(all_vars, *args):
    wec_types = ['point absorber'] #,'attenuator','terminator','point_absorber_RM3']

    # design variables
    x_name = ['pos_env']
    x = OpData(x_name)

    # parameters
    param_name = ['x_wec','x_type_wec', 'x_pen', 'gis_handler']
    param = OpData(param_name)

    # WEC and Pen Parameters are defined by optimal results obtained by running "run_sim_wec" 
    param.nom_dict['capture_width']=  np.NaN    #[m]
    param.nom_dict['pen_diameter']=   21.4     #[m]
    param.nom_dict['pen_height']=     19.3     #[m]
    param.nom_dict['stock_density']=  20       #[kg/m^3]

    if 'handler' in args[0]:
        param.nom_dict['handler'] = args[0]['handler']
    else:
        print('GIS handler is needed!')
        exit()

    #optimization
    res={}
    res_best={}
    init_flag = 1

    for i in range(len(wec_types)):
        param.nom_dict['wec_type'] = wec_types[i]
        
        res, op_obj, p = optimization.run_optimization(x.name, x.nom0, param.name, param.nom_dict, all_vars, max_iter)
        
        if init_flag:
            x_init, p_init = x , p
            res_best = copy.deepcopy(res)
            p_best = p.nom_dict
        elif (res.success) and (res.fun < res_best.fun):
            res_best = copy.deepcopy(res)
            p_best = p.nom_dict
        
        init_flag = 0
    
    return x_init, p_init, x, res_best, p_best, op_obj   

def env_bruteforce(all_vars, *args):
    wec_types = ['point absorber'] #,'attenuator','terminator','point_absorber_RM3']

    # design variables
    x_name = ['pos_env']
    x = OpData(x_name)

    # parameters
    param_name = ['x_wec','x_type_wec', 'x_pen', 'gis_handler']
    param = OpData(param_name)

    # WEC and Pen Parameters are defined by optimal results obtained by running "run_sim_wec" 
    param.nom_dict['capture_width']=  np.NaN    #[m]
    param.nom_dict['pen_diameter']=   21.4     #[m]
    param.nom_dict['pen_height']=     19.3     #[m]
    param.nom_dict['stock_density']=  20       #[kg/m^3]

    print(x.bnds)

    if 'grid_resolution' in args[0]:
        step_size = args[0]['grid_resolution']
    else:
        step_size = 1

    if 'handler' in args[0]:
        handler = args[0]['handler']
        param.nom_dict['handler'] = args[0]['handler']
    else:
        print('GIS handler is needed!')
        exit()

    lat_grid = np.arange(x.bnds[0][0], x.bnds[0][1] + step_size, step_size)
    long_grid = np.arange(x.bnds[1][0], x.bnds[1][1] + step_size, step_size)

    #brute force
    res={}
    res_best={}
    init_flag = 1

    for i in range(len(wec_types)):
        param.nom_dict['wec_type'] = wec_types[i]

        # fill default parameters
        p = optimization.argument_fun(x.name, param.name, param.nom_dict, all_vars)

        for pos_lat in (lat_grid):
            for pos_long in (long_grid):
                gis_data = handler.query(pos_long, pos_lat)
                valid_point = gis_data["ok-conditions"].bool() #and gis_data["ok-scope"].bool() and gis_data["ok-conflicts"].bool()
                print(valid_point, 'lat=', pos_lat, 'long=', pos_long)
                if valid_point:
                    x.nom_dict['pos_lat'] = pos_lat
                    x.nom_dict['pos_long'] = pos_long
                    aqua_obj = Aqua_Obj(x.nom0, x.name, p.nom_dict) 
                    if init_flag:
                        x_init, p_init = copy.deepcopy(x), copy.deepcopy(p)
                        res_best = copy.deepcopy(aqua_obj)
                        p_best = p.nom_dict
                    elif (aqua_obj.cost_per_yield < res_best.cost_per_yield):
                        res_best = copy.deepcopy(aqua_obj)
                        p_best = p.nom_dict
                    init_flag = 0
    
    return x_init, p_init, x, res_best, p_best, aqua_obj   

all_vars = ['x_wec','x_type_wec','x_pen','p_pen','x_env','p_env','p_wec','p_fish_salmon','pos_env', 'gis_handler', 'p_vessel']

conditions = {'bathymetry [m]': '../data/Bathymetry (m).tif',
              'current [m/s]': '../data/Surface Current (m-s).tif',
              'oxygen [mg/l]': '../data/Surface Oxygen (mg-l).tif',
              'salinity [PSU]': '../data/Surface Salinity (PSU).tif',
              'temperature [°C]': '../data/Surface Temperature (C).tif',
              'period [s]': '../data/Wave Energy Period (s).tif',
              'power [kW/m]': '../data/Wave Power Density (kW-m).tif',
              'height [m]': '../data/Wave Significant Height (m).tif',
              'distance to port [m]': '../data/Distance to Port (m).tif',
              'distance to shore [m]': '../data/Distance to Shore (m).tif'}

conflicts = {'wind lease': '../data/Potential Wind Lease Areas.geojson',
            'wind planning': '../data/Wind Planning Areas.geojson',
            'block island': '../data/Block Island Renewable Energy Zone.geojson',
            'ma wind': '../data/MA Wind Energy Areas.geojson',
            'shipping': '../data/Shipping Lanes.geojson',
            'submarine': '../data/Submarine Transit Lanes.geojson',
            'torpex': '../data/Cape Cod TORPEX.geojson',
            'danger zones': '../data/Danger Zones and Restricted Areas.geojson'}

waters = "../data/Northeast State and Federal Waters.geojson"

handler = GISHandler(conditions, conflicts, waters)

args = {}
args['handler'] = handler


x_init, p_init, x, res_best, p_best, op_obj = env_opt(all_vars, args)
init_result(x_init.nom0, x_init.name, p_init.nom_dict)
optimize_result(x.name, x.list, x.unit, res_best, p_best)

'''
args['grid_resolution'] = .1
x_init, p_init, x, res_best, p_best, op_obj = env_bruteforce(all_vars, args)
init_result(x_init.nom0, x_init.name, p_init.nom_dict)
bruteforce_result(x.name, x.list, x.unit, res_best, p_best)
'''

lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.6 long= -70.0              True
     35.55274665085769 810029.6824181076 3489.688328550192
lat= 42.5 long= -69.9              True
     31.915439313737227 810029.6824181076 3887.397689117754
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.5 long= -70.0              True
     34.91504277537878 810029.6824181076 3553.425548797829
lat= 42.6 long= -70.0              True
     35.55274665085769 810029.6824181076 3489.688328550192
lat= 42.5

"\nargs['grid_resolution'] = .1\nx_init, p_init, x, res_best, p_best, op_obj = env_bruteforce(all_vars, args)\ninit_result(x_init.nom0, x_init.name, p_init.nom_dict)\nbruteforce_result(x.name, x.list, x.unit, res_best, p_best)\n"